# Analyze Dust Bayesian Inference Data

In [ ]:
using Plots,Random,Measures,CSV,DataFrames,Statistics
gr();

include("auxilliary.jl"); include("poimcmc.jl");

In [ ]:
prm,_=data(); nbld = Int64(floor(prm[:nbld]));
println("Measured copies/mg dust:")
for i=1:nbld
    Yi = Symbol(:Y,i); val = prm[Yi];
    println("Bld $i: $val")
end

## Load the MCMC samples

In [ ]:
df = CSV.read("MCMCsmp.csv",DataFrame,header=false); sort!(df,:Column1); nsmp = size(df)[2]-1;
df[!,1] = Symbol.(df[!,1]);
println("Number of MCMC samples: $nsmp");

## Posterior distributions for key parameters
#### Parse parameters

In [ ]:
PRM = Vector{Dict{Symbol,Float64}}(undef,nsmp);
shift = 1;

for i=1:nsmp
    # load parameters
    prm,_ = rdprm(df[:,i+1],df[:,1]);
    PRM[i] = deepcopy(prm);
end 
nmax = Int64(floor(PRM[1][:nmax]));

#### Number infected

In [ ]:
n = Vector{Float64}(undef,nsmp)
for i=1:nsmp
    val = 0;
    @inbounds for j=1:nbld
        nj = Symbol(:n,j);
        val += Int64(floor(PRM[i][nj]));
    end
    n[i] = val;
end

In [ ]:
nstats = quantile(n,[0.025,0.25,0.5,0.75,0.975]);
navg = sum(n)/length(n);
println("n mean: $navg")
println("nstats $nstats")

p1 = histogram(n,normalize=:pdf,labels="",title="posterior",xlabel="number infected",ylabel="density",bins=100);
p2 = plot(n,linewidth=1,labels="",title="trace",xlabel="smp",ylabel="number infected");

plot(p1,p2,size=(600,300),margin=2mm)

#### Individual shedding curves
$\Gamma(\alpha,\beta)$ distribution hyperparameters for shedding amplitude

In [ ]:
Γₛ = Array{Float64,2}(undef,2,nsmp);
for i=1:nsmp
        Γₛ[1,i]=PRM[i][:Γα]+1e-6*(rand()-0.5);
        Γₛ[2,i]=PRM[i][:Γβ];
end

In [ ]:
# Joint distribution of Γ(α,β)-hyperparameters
println("Posterior joint distribution:")
p1 = histogram2d(Γₛ[1,:],Γₛ[2,:],xlabel="Γα",ylabel="Γβ",title="Shedding Distribution")
plot!(xformatter=(x->string(floor(x*1000)/1000)));

p2 = histogram2d(Γₛ[1,:]./Γₛ[2,:],((Γₛ[1,:]).^(0.5))./Γₛ[2,:],xlabel="Γ-mean",ylabel="Γ-stddev",title="Shedding Distribution")

lay = @layout [a b];
plot(p1,p2,layout=lay,size=(800,300),margin=3mm)

In [ ]:
savefig("abposterior.pdf")

$\Gamma(\bar{\alpha},\bar{\beta})$ hyperparam marginals

In [ ]:
p1 = histogram(Γₛ[1,:],normalize=:pdf,title="α-marginal",xlabel="value",ylabel="density",
               xlims=(0.0,quantile(Γₛ[1,:],0.99)),labels="",bins=100);
p2 = plot(Γₛ[1,:],linewidth=1,title="α-trace",xlabel="sample id",ylabel="value",labels="");
p3 = histogram(Γₛ[2,:],normalize=:pdf,title="β-marginal",xlabel="value",ylabel="density",
               xlims=(0.0,quantile(Γₛ[2,:],0.99)),labels="",bins=500);
p4 = plot(Γₛ[2,:],linewidth=1,title="β-trace",xlabel="sample id",ylabel="value",labels="");

lay = @layout [a b;c d]
plot(p1,p2,p3,p4,layout=lay,size=(800,450),margin=5mm)

In [ ]:
# Export hyperparameter posterior
hyper = ["alpha","beta"];
dfhyper = DataFrame(hcat(hyper,Γₛ));
CSV.write("abpost.csv",dfhyper,writeheader=false);

In [ ]:
p1 = histogram(Γₛ[1,:]./Γₛ[2,:],normalize=:pdf,title="Γ-mean",xlabel="value",ylabel="density",
               xlims=(0.0,quantile(Γₛ[1,:]./Γₛ[2,:],0.99)),labels="",bins=100);
p2 = plot(Γₛ[1,:]./Γₛ[2,:],linewidth=1,title="Γmean-trace",xlabel="sample id",ylabel="value",labels="");
p3 = histogram((Γₛ[1,:].^(0.5))./Γₛ[2,:],normalize=:pdf,title="Γ-stddev",xlabel="value",ylabel="density",
               xlims=(0.0,quantile(Γₛ[1,:].^(0.5)./Γₛ[2,:],0.99)),labels="",bins=100);
p4 = plot((Γₛ[1,:].^(0.5))./Γₛ[2,:],linewidth=1,title="Γstddev-trace",xlabel="sample id",ylabel="value",labels="");

lay = @layout [a b;c d]
plot(p1,p2,p3,p4,layout=lay,size=(800,450),margin=5mm)

$A$ amplitude of shedding

In [ ]:
A = Array{Float64,1}(undef,nmax*nsmp);
pos = 0;
for i=1:nsmp
    n = 0;
    @inbounds for j=1:nbld
        nj = Symbol(:n,j);
        n += PRM[i][nj] |> floor |> Int64;
    end
    ram = Vector{Float64}(undef,n)
    
    for j=1:n
        Anow = Symbol(:A,j);
        ram[j] = PRM[i][Anow];
    end
    
    A[pos+1:pos+n]=ram;
    pos += n;
end
A = A[1:pos];

In [ ]:
p1 = histogram(A,normalize=:pdf,labels="",title="posterior",xlabel="Amplitude peak",ylabel="density",
               xlims=(0.0,quantile(A,0.99)));
#p2 = plot(A,linewidth=1,labels="",title="trace",xlabel="smp",ylabel="Amplitude peak");

#plot(p1,p2,size=(600,300),margin=2mm)
maxA = maximum(A);
println("Max shedding amplitude: $maxA")
plot(p1,size=(300,300),margin=2mm)

$A_x$ position of peak

In [ ]:
Aₓ = Array{Float64,1}(undef,nmax*nsmp);
pos = 0;
for i=1:nsmp
    n = 0;
    @inbounds for j=1:nbld
        nj = Symbol(:n,j);
        n += PRM[i][nj] |> floor |> Int64;
    end
    ram = Vector{Float64}(undef,n)
    
    for j=1:n
        Anow = Symbol(:Aₓ,j);
        ram[j] = PRM[i][Anow];
    end
    
    Aₓ[pos+1:pos+n]=ram;
    pos += n;
end

In [ ]:
p1 = histogram(Aₓ,normalize=:pdf,labels="",title="posterior",xlabel="position of peak",ylabel="density",bins=100);
#p2 = plot(Aₓ,linewidth=1,labels="",title="trace",xlabel="smp",ylabel="position of peak");

#plot(p1,p2,size=(600,300),margin=2mm)
plot(p1,size=(300,300),margin=2mm)

L duration of shedding by infectious individual

In [ ]:
L = Array{Float64,1}(undef,nmax*nsmp);
pos = 0;
for i=1:nsmp
    n = 0;
    @inbounds for j=1:nbld
        nj = Symbol(:n,j);
        n += PRM[i][nj] |> floor |> Int64;
    end
    ram = Vector{Float64}(undef,n)
    
    for j=1:n
        Lnow = Symbol(:L,j);
        ram[j] = PRM[i][Lnow];
    end
    
    L[pos+1:pos+n]=ram;
    pos += n;
end
L = L[1:pos];

In [ ]:
p1 = histogram(L,normalize=:pdf,labels="",title="posterior",xlabel="length of shedding",ylabel="density",bins=100);
#p2 = plot(L,linewidth=1,labels="",title="trace",xlabel="smp",ylabel="length of shedding");

#plot(p1,p2,size=(600,300),margin=2mm)
plot(p1,size=(300,300),margin=2mm)

$t_i$ individual infection times

In [ ]:
t₀ = Array{Float64,1}(undef,nmax*nsmp);
pos = 0;
for i=1:nsmp
    n = 0;
    @inbounds for j=1:nbld
        nj = Symbol(:n,j);
        n += PRM[i][nj] |> floor |> Int64;
    end
    ram = Vector{Float64}(undef,n)
    
    for j=1:n
        t₀now = Symbol(:t,j);
        ram[j] = PRM[i][t₀now];
    end
    
    t₀[pos+1:pos+n]=ram;
    pos += n;
end
t₀ = t₀[1:pos];

In [ ]:
p1 = histogram(t₀,normalize=:pdf,labels="",title="posterior",xlabel="infection times",ylabel="density",bins=100);
#p2 = plot(t₀,linewidth=1,labels="",title="trace",xlabel="smp",ylabel="infection times");

#plot(p1,p2,size=(600,300),margin=2mm)
plot(p1,size=(300,300),margin=2mm)

#### individual times they took up residence and left the building

In [ ]:
tₑ = Array{Float64,1}(undef,nmax*nsmp);
pos = 0;
for i=1:nsmp
    n = 0;
    @inbounds for j=1:nbld
        nj = Symbol(:n,j);
        n += PRM[i][nj] |> floor |> Int64;
    end
    ram = Vector{Float64}(undef,n)
    
    for j=1:n
        tₑnow = Symbol(:te,j);
        ram[j] = PRM[i][tₑnow];
    end
    
    tₑ[pos+1:pos+n]=ram;
    pos += n;
end
tₑ = tₑ[1:pos];

In [ ]:
p1 = histogram(tₑ,normalize=:pdf,labels="",title="posterior",xlabel="enter building times",ylabel="density",bins=100);
#p2 = plot(t₀,linewidth=1,labels="",title="trace",xlabel="smp",ylabel="infection times");

#plot(p1,p2,size=(600,300),margin=2mm)
plot(p1,size=(300,300),margin=2mm)

In [ ]:
tℓ = Array{Float64,1}(undef,nmax*nsmp);
pos = 0;
for i=1:nsmp
    n = 0;
    @inbounds for j=1:nbld
        nj = Symbol(:n,j);
        n += PRM[i][nj] |> floor |> Int64;
    end
    ram = Vector{Float64}(undef,n)
    
    for j=1:n
        tℓnow = Symbol(:tℓ,j);
        ram[j] = PRM[i][tℓnow];
    end
    
    tℓ[pos+1:pos+n]=ram;
    pos += n;
end
tℓ = tℓ[1:pos];

In [ ]:
p1 = histogram(tℓ,normalize=:pdf,labels="",title="posterior",xlabel="exit building times",ylabel="density",bins=100);
#p2 = plot(t₀,linewidth=1,labels="",title="trace",xlabel="smp",ylabel="infection times");

#plot(p1,p2,size=(600,300),margin=2mm)
plot(p1,size=(300,300),margin=2mm)

In [ ]:
# Joint distribution of tₑ,tℓ
println("Posterior joint distribution of enter exit times:")
histogram2d(tₑ,tℓ,xlabel="tₑ",ylabel="tℓ",title="Building Enter/Exit Distribution")

In [ ]:
# Joint distribution of t₀,tₑ
println("Posterior joint distribution of time infected and enter:")
histogram2d(t₀,tₑ,xlabel="t₀",ylabel="tₑ",title="Building Inf/Enter Distribution",margin=2mm)

#### Poisson mean for total surviving virus shed 

In [ ]:
Pμ = Matrix{Float64}(undef,nsmp,nbld);
λval = Vector{Float64}(undef,nmax);
for i=1:nsmp
    shedλ!(PRM[i];λval=λval);
    pos = 0;
    for j=1:nbld
        nj = Symbol(:n,j);
        n = Int64(floor(PRM[i][nj]));
        val = 0.0;
        for k=1:n
            val += λval[pos+k]
        end
        Pμ[i,j] = val;
        pos += n;
    end
end

In [ ]:
Plts = Matrix{Any}(undef,nbld,2);
for i=1:nbld
    Plts[i,1] = histogram(Pμ[:,i],normalize=:pdf,labels="",title="posterior",xlabel="mean virus detected",ylabel="density",
               xlims=(0.00,quantile(Pμ[:,i],0.99)),bins=100);
    Plts[i,2] = plot(Pμ[:,i],linewidth=1,labels="",title="trace",xlabel="smp",ylabel="mean virus detected");
end

In [ ]:
lay = @layout [a b];
plot(Plts[1,1],Plts[1,2],size=(600,300),margin=2mm)